In [1]:
import arviz as az
import pandas as pd
import numpy as np
import pymc as pm
#import bambi as bmb
import xarray as xr
import random
import os
from sklearn.preprocessing import StandardScaler, LabelEncoder, FunctionTransformer
from scipy import stats

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
sns.set_context('paper')
sns.set_style('ticks')
plt.rc("axes.spines", top=False, right=False)
with matplotlib.style.context('arviz-doc'):
    az_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
sns.set_palette(az_colors)

def origo(ax=None):
    if ax is None:
        ax = plt.gca()
    x, y = ax.get_xlim(), ax.get_ylim()
    minx, maxx = min(x), max(x)
    miny, maxy = min(y), max(y)
    plt.ylim(min(0, miny), max(0, maxy))
    plt.xlim(min(0, minx), max(0, maxx))

class columns:
    def __rlshift__(self, df):
        "Left align columns of data frame: df << left()"
        left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
        left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector = 'th', props=[('text-align', 'left')])])
        display(left_aligned_df)
        

In [2]:
# scale = 0.35
# d = dict([(k, v*scale) for (k, v) in sns.plotting_context().items() if v is not None])
# d['figure.figsize'] = [1.8, 1.3]
# matplotlib.rcParams.update(d)
# make context for with statement:
# #matplotlib.style.context(d)

In [3]:
# plt.plot(np.random.random(10), np.random.random(10)-10)
# origo()

In [4]:
#Disable annoying "UserWarning: The figure layout has changed to tight" message
import warnings
warnings.filterwarnings('ignore', category=UserWarning
                        # , module='seaborn'
                       )
warnings.filterwarnings('ignore', category=FutureWarning,
                        # module='matplotlib'
                       )

# Model comparison

Turn HDF5 Series back into arviz `ELPDData`:

In [67]:
from arviz.stats import ELPDData
loo_hierarchical = ELPDData(pd.read_hdf('loo_hierarchical.h5'))
loo_parallel = ELPDData(pd.read_hdf('loo_parallel.h5'))

Computed from 800 posterior samples and 2940166 observations log-likelihood matrix.

         Estimate       SE
elpd_loo -4122802.30  5102.07
p_loo      286.87        -

There has been a warning during the calculation. Please check the results.
------

Pareto k diagnostic values:
                           Count   Pct.
(-Inf, 0.5]   (good)     2938948  100.0%
 (0.5, 0.7]   (ok)          1202    0.0%
   (0.7, 1]   (bad)           16    0.0%
   (1, Inf)   (very bad)       0    0.0%

ArviZ includes two convenience functions to help compare LOO for different models. The first of these functions is compare, which computes LOO (or WAIC) from a set of traces and models and returns a DataFrame.

In [ ]:
# these should ofcaues be different models...
loo_compare = az.compare({"hierarchical": loo_hierarchical, 
                           "parallel": loo_parallel})
loo_compare

We have many columns, so let’s check out their meaning one by one:

1. The index is the names of the models taken from the keys of the dictionary passed to compare(.).
2. rank, the ranking of the models starting from 0 (best model) to the number of models.
3. loo, the values of LOO (or WAIC). The DataFrame is always sorted from best LOO/WAIC to worst.
4. p_loo, the value of the penalization term. We can roughly think of this value as the estimated effective number of parameters (but do not take that too seriously).
5. d_loo, the relative difference between the value of LOO/WAIC for the top-ranked model and the value of LOO/WAIC for each model. For this reason we will always get a value of 0 for the first model.
6. weight, the weights assigned to each model. These weights can be loosely interpreted as the probability of each model being true (among the compared models) given the data.
se, the standard error for the LOO/WAIC computations. The standard error can be useful to assess the uncertainty of the LOO/WAIC estimates. By default these errors are computed using stacking.
7. dse, the standard errors of the difference between two values of LOO/WAIC. The same way that we can compute the standard error for each value of LOO/WAIC, we can compute the standard error of the differences between two values of LOO/WAIC. Notice that both quantities are not necessarily the same, the reason is that the uncertainty about LOO/WAIC is correlated between models. This quantity is always 0 for the top-ranked model.
8. warning, If True the computation of LOO/WAIC may not be reliable.
9. loo_scale, the scale of the reported values. The default is the log scale as previously mentioned. Other options are deviance – this is the log-score multiplied by -2 (this reverts the order: a lower LOO/WAIC will be better) – and negative-log – this is the log-score multiplied by -1 (as with the deviance scale, a lower value is better).

The second convenience function takes the output of compare and produces a summary plot in the style of the one used in the book Statistical Rethinking by Richard McElreath (check also this port of the examples in the book to PyMC).

In [ ]:
az.plot_compare(loo_compare, insample_dev=False);

The empty circle represents the values of LOO and the black error bars associated with them are the values of the standard deviation of LOO.

The value of the highest LOO, i.e the best estimated model, is also indicated with a vertical dashed grey line to ease comparison with other LOO values.

For all models except the top-ranked one we also get a triangle indicating the value of the difference of WAIC between that model and the top model and a grey error bar indicating the standard error of the differences between the top-ranked WAIC and WAIC for each model.

**Interpretation**

Though we might expect the hierarchical model to outperform a complete pooling model, there is little to choose between the models in this case, given that both models gives very similar values of the information criteria. This is more clearly appreciated when we take into account the uncertainty (in terms of standard errors) of LOO and WAIC.

[https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/model_comparison.html](https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/model_comparison.html)

In [ ]:
az.plot_elpd(
    {"hierarchical": elpd_hierarchical, 
     "parallel": elpd_parallel},
    xlabels=True
)

## Inference

In [ ]:
idata_hierarchical.posterior.to_dataframe().head()

In [ ]:
az.extract(idata_hierarchical, num_samples=20)

In [ ]:
data[['full_species', 'NE_MEAN']].drop_duplicates().set_index('full_species').loc[t.Species, 'NE_MEAN']

In [ ]:
d = idata.posterior.mean(dim=['chain', 'draw'])
sns.barplot(data=pd.DataFrame(dict(x=d.Species, y=d.b)), x='x', y='y')

In [ ]:
idata.groups()

In [ ]:
sns.set_context('paper')
az.plot_bf(idata.posterior, var_name="h1") ;
#sns.despine()

In [ ]:
az.plot_forest(idata.posterior, var_names=['h0', 'h1']) ;

In [ ]:
az.plot_posterior(idata, var_names=['a', 'b'], figsize=(16, 15))
plt.tight_layout()

In [ ]:
az.plot_posterior(idata, var_names=['a', 'b'], combine_dims={'Species'}, figsize=(6, 5)) ;

In [ ]:
az.plot_forest(idata.posterior, var_names=['g0', 'g1', 'h0', 'h1'], ess=True, figsize=(6, 5)) ;

In [ ]:
az.plot_forest(idata.posterior, var_names=['b'], ess=True, figsize=(15, 100)) ;

In [ ]:
import arviz as az
import numpy as np
import timeit
import dask

from arviz.utils import conditional_jit, Dask

In [ ]:
# optional imports
from dask.distributed import Client
from dask.diagnostics import ResourceProfiler

from bokeh.resources import INLINE
import bokeh.io

bokeh.io.output_notebook(INLINE)

%reload_ext memory_profiler

> ResourceProfiler() and Client are optional. They are only used for the visualizing and profiling the dask enabled methods. ArviZ-Dask integration can be used without using these objects.

In [ ]:
client = Client(threads_per_worker=4, n_workers=1, memory_limit="1.2GB")
client

In [ ]:
array_size = 250_000_000

In [ ]:
%%memit 
data = np.random.randn(array_size)
np.var(data, ddof=1)
del data

In [ ]:
%memit data = dask.array.random.normal(size=array_size, chunks="auto")
data

In [ ]:
var = dask.array.var(data, ddof=1)
var.visualize()

In [ ]:
with ResourceProfiler(dt=0.25) as rprof:
    var.compute()

rprof.visualize();